In [4]:
# import tweepy
import os
import glob
import time
import numpy as np
import pandas as pd
import re

# key
# consumer_key = 'D4OAzhqT7NNeeCa9R7h0Hxzx3'
# consumer_secret = 'Py8P1k5jxdPEvNOb64WUXh8b10sfDL5qr60krVLNOOLEXauZti'
# access_token = '1309887734555668481-fHCklZt9pzFw43tUdBT2pKlb7ktNDD'
# access_secret = 'ETRRfhb3aVxcAFcUUKh9ctSYjgDOHbXmp3Iwns1q562cu'
 
# Twitterオブジェクト
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True)

In [5]:
division = input('ディビジョンを選択: ')
# s_round = '2021-22_regular'
# s_round = '2021-22_finalstage'
s_round = '2021-22_final'
# s_round = '2021-22_challenge'
div_dir = '{}/{}'.format(s_round, division)

daily_all = pd.read_csv('{}/daily_all.csv'.format(div_dir),encoding='cp932')
# daily_all
day = '2022-04-10'
today_data = daily_all[daily_all['試合日']==day]
# today_data = daily_all
# today_data.head()
column_list = ['総得点', 'アタック決定率','アタック打数', 'バックアタック決定率', 
               'ブロックセット平均', 'サーブ得点', 'サーブ効果率', 'サーブレシーブ成功率']
# stats = np.random.choice(stats_list)
for column in column_list:
#     print(stats)
    if column == '総得点':
        reg_judge = today_data['出場数']>=2
        pattern = ['出場数2セット', 2]
    elif column == 'アタック打数':
        reg_judge = today_data['出場数']>=2
        pattern = ['出場数2セット', 0]
    elif column == 'アタック決定率':
        reg_judge = today_data['アタック打数']>=10
        pattern = ['打数10本', 1]
    elif column == 'バックアタック決定率':
        reg_judge = today_data['バックアタック打数']>=5
        pattern = ['打数5本', 1]
    elif column == 'ブロックセット平均':
        reg_judge = today_data['出場数']>=2
        pattern = ['出場数2セット', 0]
    elif column == 'サーブ得点':
        reg_judge = today_data['サーブ打数']>=5
        pattern = ['サーブ打数5本', 2]
    elif column == 'サーブ効果率':
        reg_judge = today_data['サーブ打数']>=5
        pattern = ['サーブ打数5本', 1]
    else:
        reg_judge = today_data['受数']>=5
        column = 'サーブレシーブ成功率'
        pattern = ['受数5本', 1]
#     print(column)

    for i in range(3):
        stats_df = today_data[reg_judge][['名前', 'チーム', '背番号', '出場数', column]]
        if stats_df[column].dtypes == object:
            stats_df[column] = stats_df[column].str.replace('-', '') 
            stats_df[column] = stats_df[column].apply(pd.to_numeric)
        stats_df = stats_df.sort_values(column, ascending=False).head()
#         print(stats_df)
        if len(stats_df) >= 3:
            break
    # try:
    # stats_df = stats_df.to_dict()
#     print(stats_df)
    if 'm' in division:
        division_j = 'V' + division[1] + '男子'
    else:
        division_j = 'V' + division[1] + '女子'
    
    p = re.compile('[\u0000-\u007F]+')
    messages = ''
    messages = '【#{}】'.format(division_j)
    messages += '#ファイナル 1日目の成績\n'
#     messages += '#チャレンジマッチ ２日目の成績\n'
#     messages += '本日({})の成績\n'.format(day)
    messages += '【{0}】ランキング({1}以上)\n'.format(column, pattern[0])
    # messages += 'ディビジョン: #{}\n'.format(division_j)
#     print(len(stats_df))
    for i in range(len(stats_df)):
        if pattern[1]==1:
            unit = '%'
        elif pattern[1]==2:
            unit = '点'
        else:
            unit = '本'
        
        if i >= 1 and stats_df[column].iloc[i-1] == stats_df[column].iloc[i]:
            tie = '   '
            rank = ''
        else:
            tie = ''
            rank = '{}位'.format(i+1)
        message = '{0}{5} #{1} ({2}, {3}{4})\n'.format(rank, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i], unit, tie)
        m_len = messages + message
        e_len = len(''.join(p.findall(m_len)))
        tweet_len = len(m_len) - (e_len/2)
#         print(tweet_len)
        if tweet_len <= 140 and stats_df[column].iloc[i] != 0:
            messages += message
        else:
            break
    print(messages)
    if len(stats_df) == 0:
        print('not tweet')
    else:
#         api.update_status(messages)
        pass
    # except:
    #     pass

ディビジョンを選択:  v1_m


【#V1男子】#ファイナル 1日目の成績
【総得点】ランキング(出場数2セット以上)
1位 #ムセルスキードミトリー (サントリーサンバーズ, 21点)
2位 #クレクバルトシュ (ウルフドッグス名古屋, 20点)
3位 #高梨健太 (ウルフドッグス名古屋, 13点)
4位 #山崎彰都 (ウルフドッグス名古屋, 11点)

【#V1男子】#ファイナル 1日目の成績
【アタック決定率】ランキング(打数10本以上)
1位 #クレクバルトシュ (ウルフドッグス名古屋, 67.9%)
2位 #ムセルスキードミトリー (サントリーサンバーズ, 61.3%)
3位 #山崎彰都 (ウルフドッグス名古屋, 55.6%)
4位 #柳田将洋 (サントリーサンバーズ, 50.0%)

【#V1男子】#ファイナル 1日目の成績
【アタック打数】ランキング(出場数2セット以上)
1位 #ムセルスキードミトリー (サントリーサンバーズ, 31本)
2位 #クレクバルトシュ (ウルフドッグス名古屋, 28本)
3位 #山崎彰都 (ウルフドッグス名古屋, 18本)
4位 #高梨健太 (ウルフドッグス名古屋, 17本)

【#V1男子】#ファイナル 1日目の成績
【バックアタック決定率】ランキング(打数5本以上)
1位 #クレクバルトシュ (ウルフドッグス名古屋, 78.6%)
2位 #ムセルスキードミトリー (サントリーサンバーズ, 64.3%)
3位 #高梨健太 (ウルフドッグス名古屋, 40.0%)

【#V1男子】#ファイナル 1日目の成績
【ブロックセット平均】ランキング(出場数2セット以上)
1位 #傳田亮太 (ウルフドッグス名古屋, 0.67本)
    #永露元稀 (ウルフドッグス名古屋, 0.67本)
    #高梨健太 (ウルフドッグス名古屋, 0.67本)
    #彭世坤 (サントリーサンバーズ, 0.67本)

【#V1男子】#ファイナル 1日目の成績
【サーブ得点】ランキング(サーブ打数5本以上)
1位 #傳田亮太 (ウルフドッグス名古屋, 3点)
    #高梨健太 (ウルフドッグス名古屋, 3点)
3位 #クレクバルトシュ (ウルフドッグス名古屋, 1点)
    #山崎彰都 (ウルフドッグス名古屋, 1点)

【#V1男子】#ファイナル 1日目の成績
【サーブ効果率

In [141]:
teams = list(today_data.drop_duplicates(keep='first', subset=["チーム"])["チーム"].values)
not_tweet_list = []
# print(teams)
# today_team_data = today_data[today_data["チーム"]==teams[0]]
# today_team_data
for team in teams:
    today_team_data = today_data[today_data["チーム"]==team]
    message = ''
    # message += "【ファイナル】"
    # message += "【チャレンジマッチ女子】"
    message += "【#{}】#{}\n".format(division_j,team)
#     message += "本日({})のスタッツリーダー\n".format(day)
    message += "1日目のスタッツリーダー\n"
    column_list = ["総得点", "アタック決定率", "ブロックセット平均", "サーブ効果率", "サーブレシーブ成功率"]
    for column in column_list:
        # print(stats)
        if column == '総得点':
            reg_judge = today_team_data['出場数']>=2
            unit = "点"
        elif column == 'アタック決定率':
            reg_judge = today_team_data['アタック打数']>=5
            unit = '%'
        # elif column == 'バックアタック決定率':
        #     reg_judge = today_data['バックアタック打数']>=5
        elif column == 'ブロックセット平均':
            reg_judge = today_team_data['出場数']>=2
            unit = "本"
        elif column == 'サーブ効果率':
            reg_judge = today_team_data['サーブ打数']>=5
            unit = '%'
        else:
            reg_judge = today_team_data['受数']>=5
            unit = '%'
        stats_df = today_team_data[reg_judge][['名前', 'チーム', '背番号', '出場数', column]]
        if stats_df[column].dtypes == object:
            stats_df[column] = stats_df[column].str.replace('-', '') 
            stats_df[column] = stats_df[column].apply(pd.to_numeric)
        stats_df = stats_df.sort_values([column, "出場数"], ascending=False).head()
        if  not np.isnan(stats_df.iloc[0, 4]):
            message += '{} #{} #{} {}{}'.format(column, stats_df.iloc[0, 2], stats_df.iloc[0, 0], stats_df.iloc[0, 4], unit)
        j = 0
        for i in range(1, len(stats_df)):
            # print(stats_df.iloc[0, 4], stats_df.iloc[i, 4])
            if stats_df.iloc[0, 4] == stats_df.iloc[i, 4]:
                j += 1
        # print(j)
        if j > 0:
            message += "(他同率{}人)".format(j)
        if  not np.isnan(stats_df.iloc[0, 4]):
            message += '\n'
    print(message)
    p = re.compile('[\u0000-\u007F]+')
    e_len = len(''.join(p.findall(message)))
    tweet_len = len(message) - (e_len/2)
    if s_round == '2021-22_finalstage':
        message += '\n#ファイナルステージ'
#     print(tweet_len)
    if tweet_len <= 140:
        print(tweet_len)
#         api.update_status(message)
    else:
        print("文字数超過")
        not_tweet_list.append(team)
# stats_df
print(not_tweet_list)

【#V1男子】#ウルフドッグス名古屋
1日目のスタッツリーダー
総得点 #3 #クレクバルトシュ 20点
アタック決定率 #3 #クレクバルトシュ 67.9%
ブロックセット平均 #11 #傳田亮太 0.67本(他同率2人)
サーブ効果率 #11 #傳田亮太 34.1%
サーブレシーブ成功率 #26 #山崎彰都 67.6%

128.5
【#V1男子】#サントリーサンバーズ
1日目のスタッツリーダー
総得点 #13 #ムセルスキードミトリー 21点
アタック決定率 #13 #ムセルスキードミトリー 61.3%
ブロックセット平均 #6 #彭世坤 0.67本
サーブ効果率 #13 #ムセルスキードミトリー 8.3%
サーブレシーブ成功率 #1 #鶴田大樹 66.7%

134.5
[]


In [38]:
# time_start_tables = pd.DataFrame(data=0, index=['開幕前'], columns=time_tables.columns)
# time_start_tables['Date'] = '開幕前'
# time_start_tables

In [142]:
yearly_all = pd.read_csv('{}/yearly_all.csv'.format(div_dir),encoding='cp932')
# game_dict = {'v1_m': 36, 'v2_m': 28, 'v3_m':30, 'v1_w':33, 'v2_w':10}
# game_dict = {'v2_w':3}
game_dict = {'v1_m': 1, 'v1_w': 1}
# game_dict = {'v1_m':2, 'v2_m':2, 'v1_w':2}
n = game_dict[division]
stats_list = ['total', 'at_attack','attack_k', 'b_attack_k', 'block', 'surve_k', 'surve_eff', 'cut']
# stats = np.random.choice(stats_list)
for stats in stats_list:
#     print(stats)
    if stats == 'total':
        reg_judge = yearly_all['出場数']>=n*2
        column = '総得点'
        pattern = ['出場数{}セット'.format(n*2), 2]
    elif stats == 'at_attack':
        reg_judge = yearly_all['出場数']>=n*2
        column ='アタック打数'
        pattern = ['出場数{}セット'.format(n*2), 0]
    elif stats == 'attack_k':
        reg_judge = yearly_all['アタック打数']>=n*10
        column = 'アタック決定率'
        
        pattern = ['打数{}本'.format(n*10), 1]
    elif stats == 'b_attack_k':
        reg_judge = yearly_all['バックアタック打数']>=n*5
        column = 'バックアタック決定率'
        pattern = ['打数{}本'.format(n*5), 1]
    elif stats == 'block':
        reg_judge = yearly_all['出場数']>=n*2
        column = 'ブロックセット平均'
        pattern = ['出場数{}セット'.format(n*2), 0]
    elif stats == 'surve_k':
        reg_judge = yearly_all['サーブ打数']>=n*5
        column = 'サーブ得点'
        pattern = ['サーブ打数{}本'.format(n*5), 2]
    elif stats == 'surve_eff':
        reg_judge = yearly_all['サーブ打数']>=n*5
        column = 'サーブ効果率'
        pattern = ['サーブ打数{}本'.format(n*5), 1]
    else:
        reg_judge = yearly_all['受数']>=n*5
        column = 'サーブレシーブ成功率'
        pattern = ['受数{}本'.format(n*5), 1]
#     print(column)

    for i in range(3):
        stats_df = yearly_all[reg_judge][['名前', 'チーム', '背番号', '出場数', column]]
        if stats_df[column].dtypes == object:
            stats_df[column] = stats_df[column].str.replace('-', '') 
            stats_df[column] = stats_df[column].apply(pd.to_numeric)
        stats_df = stats_df.sort_values(column, ascending=False).head()
#         print(stats_df)
        if len(stats_df) >= 3:
            break
    # try:
    # stats_df = stats_df.to_dict()

#     print(stats_df)
    if 'm' in division:
        div = 'V' + division[1] + '男子'
    else:
        div = 'V' + division[1] + '女子'


    p = re.compile('[\u0000-\u007F]+')
    if s_round == '2021-22_finalstage':
        season = 'ファイナルステージ'
    else:
        season = '今季'
#     messages = '【#{}】{}'.format(division_j, season)
    messages = ''
    messages += '【#{}】#チャレンジマッチ 通算\n'.format(division_j)
#     messages += '通算成績(※{}時点)\n'.format(day)
#     messages += '通算成績(※今シーズン終了時点)\n'
#     messages += '(ゴールデンセットを含む)\n'
    messages += '【{0}】ランキング({1}以上)\n'.format(column, pattern[0])
#     print(len(stats_df))
    for i in range(len(stats_df)):
        if pattern[1]==1:
            unit = '%'
        elif pattern[1]==2:
            unit = '点'
        else:
            unit = '本'
        
        if i >= 1 and stats_df[column].iloc[i-1] == stats_df[column].iloc[i]:
            tie = '   '
            rank = ''
        else:
            tie = ''
            rank = '{}位'.format(i+1)
        message = '{0}{5} #{1} ({2}, {3}{4})\n'.format(rank, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i], unit, tie)
        m_len = messages + message
        e_len = len(''.join(p.findall(m_len)))
        tweet_len = len(m_len) - (e_len/2)
        # print(tweet_len)
        if tweet_len <= 140:
            messages += message
        else:
            break
    print(messages)
#     api.update_status(messages)
    # except:
    #     pass

【#V1男子】#チャレンジマッチ 通算
【総得点】ランキング(出場数2セット以上)
1位 #ムセルスキードミトリー (サントリーサンバーズ, 21点)
2位 #クレクバルトシュ (ウルフドッグス名古屋, 20点)
3位 #高梨健太 (ウルフドッグス名古屋, 13点)
4位 #山崎彰都 (ウルフドッグス名古屋, 11点)

【#V1男子】#チャレンジマッチ 通算
【アタック打数】ランキング(出場数2セット以上)
1位 #ムセルスキードミトリー (サントリーサンバーズ, 31本)
2位 #クレクバルトシュ (ウルフドッグス名古屋, 28本)
3位 #山崎彰都 (ウルフドッグス名古屋, 18本)
4位 #高梨健太 (ウルフドッグス名古屋, 17本)

【#V1男子】#チャレンジマッチ 通算
【アタック決定率】ランキング(打数10本以上)
1位 #クレクバルトシュ (ウルフドッグス名古屋, 67.9%)
2位 #ムセルスキードミトリー (サントリーサンバーズ, 61.3%)
3位 #山崎彰都 (ウルフドッグス名古屋, 55.6%)
4位 #柳田将洋 (サントリーサンバーズ, 50.0%)

【#V1男子】#チャレンジマッチ 通算
【バックアタック決定率】ランキング(打数5本以上)
1位 #クレクバルトシュ (ウルフドッグス名古屋, 78.6%)
2位 #ムセルスキードミトリー (サントリーサンバーズ, 64.3%)
3位 #高梨健太 (ウルフドッグス名古屋, 40.0%)

【#V1男子】#チャレンジマッチ 通算
【ブロックセット平均】ランキング(出場数2セット以上)
1位 #永露元稀 (ウルフドッグス名古屋, 0.67本)
    #傳田亮太 (ウルフドッグス名古屋, 0.67本)
    #高梨健太 (ウルフドッグス名古屋, 0.67本)
    #彭世坤 (サントリーサンバーズ, 0.67本)

【#V1男子】#チャレンジマッチ 通算
【サーブ得点】ランキング(サーブ打数5本以上)
1位 #傳田亮太 (ウルフドッグス名古屋, 3点)
    #高梨健太 (ウルフドッグス名古屋, 3点)
3位 #クレクバルトシュ (ウルフドッグス名古屋, 1点)
    #ムセルスキードミトリー (サントリーサンバーズ, 1点)

【#V1男子】#チャレンジマッチ 通算
【サーブ効果率

In [143]:
teams = list(yearly_all.drop_duplicates(keep='first', subset=["チーム"])["チーム"].values)
# day_m = day[-4:].replace('0', '').replace('-', '/')
day_m = day
for team in teams:
    yearly_team = yearly_all[yearly_all["チーム"]==team]
#     message = "{}通算スタッツリーダー({}時点)\nチーム: #{}\n".format(season, day_m, team)
#     message = "{}通算スタッツリーダー(最終成績)\n".format(season)
    message = "【#{}】#チャレンジマッチ 通算スタッツリーダー\n".format(division_j)
    message += "チーム: #{}\n".format(team)
    stats_list = ["総得点", "アタック決定率", "ブロックセット平均", "サーブ効果率", "サーブレシーブ成功率"]
    for stats in stats_list:
        # print(stats)
        if stats == '総得点':
            reg_judge = yearly_team['出場数']>=n
            unit = "点"
        elif stats == 'アタック決定率':
            reg_judge = yearly_team['アタック打数']>=n*10
            unit = '%'
        # elif stats == 'バックアタック決定率':
        #     reg_judge = today_data['バックアタック打数']>=n*4
        elif stats == 'ブロックセット平均':
            reg_judge = yearly_team['出場数']>=n
            unit = "本"
        elif stats == 'サーブ効果率':
            reg_judge = yearly_team['サーブ打数']>=n*5
            unit = '%'
        else:
            reg_judge = yearly_team['受数']>=n*5
            unit = '%'
        stats_df = yearly_team[reg_judge][['名前', 'チーム', '背番号', '出場数', stats]]
        if stats_df[stats].dtypes == object:
            stats_df[stats] = stats_df[stats].str.replace('-', '') 
            stats_df[stats] = stats_df[stats].apply(pd.to_numeric)
        stats_df = stats_df.sort_values([stats, "出場数"], ascending=False).head()
        message += '{} #{} #{} {}{}'.format(stats, stats_df.iloc[0, 2], stats_df.iloc[0, 0], stats_df.iloc[0, 4], unit)
        j = 0
        for i in range(1, len(stats_df)):
            # print(stats_df.iloc[0, 4], stats_df.iloc[i, 4])
            if stats_df.iloc[0, 4] == stats_df.iloc[i, 4]:
                j += 1
        # print(j)
        if j > 0:
            message += "(他同率{}人)".format(j)
        message += '\n'
    print(message)
    p = re.compile('[\u0000-\u007F]+')
    e_len = len(''.join(p.findall(message)))
    tweet_len = len(message) - (e_len/2)
#     print(tweet_len)
    if tweet_len <= 140:
        print(tweet_len)
#         api.update_status(message)
    else:
        print("文字数超過")
# stats_df

【#V1男子】#チャレンジマッチ 通算スタッツリーダー
チーム: #ウルフドッグス名古屋
総得点 #3 #クレクバルトシュ 20点
アタック決定率 #3 #クレクバルトシュ 67.9%
ブロックセット平均 #11 #傳田亮太 0.67本(他同率2人)
サーブ効果率 #11 #傳田亮太 34.1%
サーブレシーブ成功率 #26 #山崎彰都 67.6%

140.0
【#V1男子】#チャレンジマッチ 通算スタッツリーダー
チーム: #サントリーサンバーズ
総得点 #13 #ムセルスキードミトリー 21点
アタック決定率 #13 #ムセルスキードミトリー 61.3%
ブロックセット平均 #6 #彭世坤 0.67本
サーブ効果率 #13 #ムセルスキードミトリー 8.3%
サーブレシーブ成功率 #1 #鶴田大樹 66.7%

文字数超過


In [144]:
def stats_calc(df):
    df['アタック決定率'] = ((df['アタック得点'] / df['アタック打数']) * 100).round(1)
    df['アタック効果率'] = (((df['アタック得点']-df['アタック失点']) /
                     df['アタック打数']) * 100).round(1)
    df['バックアタック決定率'] = ((df['バックアタック得点'] / df['バックアタック打数']) * 100).round(1)
    df['アタックセット平均'] = (df['アタック得点'] / df['出場数']).round(2)
    df['ブロックセット平均'] = (df['ブロック得点'] / df['出場数']).round(2)
    df['サーブ効果率'] = ((df['サーブ得点'] * 100 + df['サーブ効果'] * 25 -
                    df['サーブ失点'] * 25) / df['サーブ打数']).round(1)
    df['サーブレシーブ成功率'] = (
        (df['サーブレシーブ成功・優'] * 100 + df['サーブレシーブ成功・良'] * 50) / df['受数']).round(1)
    return df

In [145]:
# division = 'v1_m'
# s_round = '2021-22_finalstage'
div_dir = '{}/{}'.format(s_round, division)
yearly_all = pd.read_csv('{}/yearly_all.csv'.format(div_dir),encoding='cp932')
yearly_team = yearly_all.groupby('チーム').sum()
yearly_team = stats_calc(yearly_team).drop(columns=['背番号','得セット','失セット', '勝敗', '出場数'])
yearly_team.to_csv('{}/yearly_team.csv'.format(div_dir),encoding='cp932')
yearly_team

,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,バックアタック打数,バックアタック得点,バックアタック失点,ブロック得点,サーブ打数,...,サーブ効果,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,総得点,バックアタック決定率,アタックセット平均,ブロックセット平均,サーブ効果率,サーブレシーブ成功率
チーム,,,,,,,,,,,,,,,,,,,,,
ウルフドッグス名古屋,71,42,6,59.2,50.7,22,14,3,7,76,...,18,53,27,11,57,63.6,1.62,0.27,11.5,61.3
サントリーサンバーズ,76,34,6,44.7,36.8,20,11,3,6,68,...,15,58,22,13,41,55.0,1.21,0.21,1.8,49.1


In [146]:
yearly_team = pd.read_csv('{}/yearly_team.csv'.format(div_dir),encoding='cp932')
yearly_team

,チーム,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,バックアタック打数,バックアタック得点,バックアタック失点,ブロック得点,...,サーブ効果,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,総得点,バックアタック決定率,アタックセット平均,ブロックセット平均,サーブ効果率,サーブレシーブ成功率
0,ウルフドッグス名古屋,71,42,6,59.2,50.7,22,14,3,7,...,18,53,27,11,57,63.6,1.62,0.27,11.5,61.3
1,サントリーサンバーズ,76,34,6,44.7,36.8,20,11,3,6,...,15,58,22,13,41,55.0,1.21,0.21,1.8,49.1


In [147]:
if 'm' in division:
    div = 'V' + division[1] + '男子'
else:
    div = 'V' + division[1] + '女子'

yearly_team = pd.read_csv('{}/yearly_team.csv'.format(div_dir),encoding='cp932')
stats_list = ['アタック決定率', 'アタック効果率', 'バックアタック決定率', 'サーブレシーブ成功率', 'サーブ効果率']
# stats = np.random.choice(stats_list)
for stats in stats_list:
#     print(stats)
    if stats in ['アタック決定率','アタック効果率','バックアタック決定率','サーブレシーブ成功率', 'サーブ効果率']:
        unit = '%'
    elif stats in ['アタックセット平均']:
        unit = '本'


    for i in range(3):
        stats_df = yearly_team[['チーム', stats]]
        if stats_df[stats].dtypes == object:
            stats_df[stats] = stats_df[stats].str.replace('-', '') 
            stats_df[stats] = stats_df[stats].apply(pd.to_numeric)
        stats_df = stats_df.sort_values(stats, ascending=False).head()
#         print(stats_df)
        if len(stats_df) >= 3:
            break
    # try:
    # stats_df = stats_df.to_dict()

#     print(stats_df)

    p = re.compile('[\u0000-\u007F]+')
    messages = ''
#     if s_round == '2021-22_finalstage':
#         messages += '【ファイナルステージ】\n'
#     messages += '【#{0}】{1}チーム通算成績(※{2}時点)\n'.format(div, season ,day_m)
#     messages += '【#{0}】{1}チーム通算成績(最終成績)\n'.format(div, season)
    messages += '【#{0}】#チャレンジマッチ チーム通算成績(最終成績)\n'.format(div)
    messages += '【{0}】ランキング\n'.format(stats)
#     print(len(stats_df))
    for i in range(len(stats_df)):
        if pattern[1]==1:
            unit = '%'
        elif pattern[1]==2:
            unit = '点'
        else:
            unit = '本'
        
        if i >= 1 and stats_df[stats].iloc[i-1] == stats_df[stats].iloc[i]:
            tie = '   '
            rank = ''
        else:
            tie = ''
            rank = '{}位'.format(i+1)
        message = '{0}{4} #{1} {2}{3}\n'.format(rank, stats_df['チーム'].iloc[i], stats_df[stats].iloc[i], unit, tie)
        m_len = messages + message
        e_len = len(''.join(p.findall(m_len)))
        tweet_len = len(m_len) - (e_len/2)
        # print(tweet_len)
        if tweet_len <= 140:
            messages += message
        else:
            break
    print(messages)
#     api.update_status(messages)
    # except:
    #     pass

【#V1男子】#チャレンジマッチ チーム通算成績(最終成績)
【アタック決定率】ランキング
1位 #ウルフドッグス名古屋 59.2%
2位 #サントリーサンバーズ 44.7%

【#V1男子】#チャレンジマッチ チーム通算成績(最終成績)
【アタック効果率】ランキング
1位 #ウルフドッグス名古屋 50.7%
2位 #サントリーサンバーズ 36.8%

【#V1男子】#チャレンジマッチ チーム通算成績(最終成績)
【バックアタック決定率】ランキング
1位 #ウルフドッグス名古屋 63.6%
2位 #サントリーサンバーズ 55.0%

【#V1男子】#チャレンジマッチ チーム通算成績(最終成績)
【サーブレシーブ成功率】ランキング
1位 #ウルフドッグス名古屋 61.3%
2位 #サントリーサンバーズ 49.1%

【#V1男子】#チャレンジマッチ チーム通算成績(最終成績)
【サーブ効果率】ランキング
1位 #ウルフドッグス名古屋 11.5%
2位 #サントリーサンバーズ 1.8%



In [37]:
team_rank = yearly_team[['アタック決定率','アタック効果率','バックアタック決定率','サーブレシーブ成功率', 'サーブ効果率']].rank(method='dense', ascending=False)
team_rank

,アタック決定率,アタック効果率,バックアタック決定率,サーブレシーブ成功率,サーブ効果率
0,1.0,1.0,1.0,2.0,1.0
1,3.0,4.0,4.0,1.0,4.0
2,2.0,2.0,2.0,3.0,3.0
3,2.0,3.0,3.0,4.0,2.0


In [38]:
yearly_team

,チーム,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,バックアタック打数,バックアタック得点,バックアタック失点,ブロック得点,...,サーブ効果,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,総得点,バックアタック決定率,アタックセット平均,ブロックセット平均,サーブ効果率,サーブレシーブ成功率
0,アイシンティルマーレ,1286,602,84,46.8,40.3,280,134,31,91,...,222,749,368,216,742,47.9,1.49,0.22,6.6,63.6
1,トヨタモビリティ東京スパークル,1058,441,90,41.7,33.2,124,50,18,56,...,141,749,408,160,527,40.3,1.41,0.18,4.4,65.2
2,近畿クラブスフィーダ,1484,655,121,44.1,36.0,206,92,19,135,...,215,956,474,221,817,44.7,1.43,0.30,4.5,61.1
3,長野GaRons,1516,669,134,44.1,35.3,274,115,47,116,...,258,969,453,216,818,42.0,1.37,0.24,6.0,57.9
